In [1]:
import numpy as np
import pandas as pd
from keras.utils import np_utils
from tqdm import tqdm
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import cv2

Using TensorFlow backend.


In [2]:
test_submit_df = pd.read_csv('//data/examples/may_the_4_be_with_u/where_am_i/img-submission.csv')
#test_submit_df.columns = ['file_name', 'label']
print(len(test_submit_df))
test_submit_df.head(5)

1500


,id,class
0,c117693e1cf24a5232090d1548cb11d4e5ea0df65680c4...,1
1,96baacc2e97886a998807ce197574821a6dc83c227c746...,1
2,f9bd26db23eb9b544ca78be79b11b4d1259e802885861d...,1
3,b42bcd8e6645fcc2ac40ee44b7dc8d74a77081d0aea7a1...,1
4,5bc53cef9168882f0ff67a81b3e7269f62b7fd5343d06d...,1


In [3]:
print("=====================start loading data=====================")
image_width = 256
image_high = 256
#image_cnn_shape = (image_width//16) * (image_high//16)
path = "//data/examples/may_the_4_be_with_u/where_am_i/"
test_path = path + "testset/"
image = cv2.imread(test_path + test_submit_df["id"][0] + ".jpg")
image = cv2.resize(image, (image_width, image_high))
#image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
test = np.expand_dims(image, axis=0)
for x in tqdm(range(1, len(test_submit_df))):
    path = test_path + test_submit_df["id"][x] + ".jpg"
    image = cv2.imread(path)
    image = cv2.resize(image, (image_width, image_high))
    #image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image = np.expand_dims(image, axis=0)
    test = np.concatenate((test,image), axis=0)
    
test_label = np.zeros(0, dtype=float)

print("\t[Info] Shape of test data=%s" % (str(test.shape)))
print("\t[Info] Shape of test_label=%s" % (str(test_label.shape)))

  3%|▎         | 46/1499 [00:00<00:03, 449.17it/s]

=====================start loading data=====================


100%|██████████| 1499/1499 [01:14<00:00, 20.04it/s]

	[Info] Shape of test data=(1500, 256, 256, 3)
	[Info] Shape of test_label=(0,)


In [4]:
# Normalization
test = test/255
print("\t[Info] xTest: %s" % (str(test.shape)))

	[Info] xTest: (1500, 256, 256, 3)


In [5]:
#tf.reset_default_graph()
#sess = tf.InteractiveSession()

# 重新定義模組
TensorFlow比較麻煩的是要用之前保存下來的模組結果，沒辦法引入架構，必須要自己重寫。

In [6]:
from __future__ import print_function
import tensorflow as tf

batch_size = 32
epochs = 70
lr = 0.0001

hidden1_neurons = 512
hidden2_neurons = 256
image_cnn_shape = (image_width//32) * (image_high//32)

#def compute_accuracy(v_xs, v_ys):
#    global prediction
#    y_pre = sess.run(prediction, feed_dict={xs: v_xs, keep_prob: 1})
#    correct_prediction = tf.equal(tf.argmax(y_pre,1), tf.argmax(v_ys,1))
#    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
#    result = sess.run(accuracy, feed_dict={xs: v_xs, ys: v_ys, keep_prob: 1})
#    return result

def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W):
    # stride [1, x_movement, y_movement, 1]
    # Must have strides[0] = strides[3] = 1
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    # stride [1, x_movement, y_movement, 1]
    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

# define placeholder for inputs to network
xs = tf.placeholder(tf.float32, [None, image_width, image_high, 3])   # 256*256*3
ys = tf.placeholder(tf.float32, [None, 15])
keep_prob = tf.placeholder(tf.float32)
x_image = tf.reshape(xs, [-1, image_width, image_high, 3])
# print(x_image.shape)  # [n_samples, 256,200,1]
#[16,256,256,3] [3, 3, 3, 32]
## conv1 layer ##
W_conv1 = weight_variable([3, 3, 3, 64]) # patch 3x3, in size 3, out size 32
b_conv1 = bias_variable([64])
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1) # output size 256*256*32

W_conv1_2 = weight_variable([3, 3, 64, 64]) # patch 3x3, in size 32, out size 32
b_conv1_2 = bias_variable([64])
h_conv1_2 = tf.nn.relu(conv2d(h_conv1, W_conv1_2) + b_conv1_2) # output size 256*256*32
h_pool1 = max_pool_2x2(h_conv1_2)                          # output size 128*128*32

## conv2 layer ##
W_conv2 = weight_variable([3, 3, 64, 128]) # patch 3x3, in size 32, out size 64
b_conv2 = bias_variable([128])
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2) # output size 128x128x64

W_conv2_2 = weight_variable([3, 3, 128, 128]) # patch 3x3, in size 32, out size 32
b_conv2_2 = bias_variable([128])
h_conv2_2 = tf.nn.relu(conv2d(h_conv2, W_conv2_2) + b_conv2_2) # output size 256*256*32

h_pool2 = max_pool_2x2(h_conv2_2)                          # output size 64x64x64
h_pool2_drop = tf.nn.dropout(h_pool2, keep_prob)

## conv3 layer ##
W_conv3 = weight_variable([3 ,3 , 128, 256]) # patch 5x5, in size 64, out size 128
b_conv3 = bias_variable([256])
h_conv3 = tf.nn.relu(conv2d(h_pool2_drop, W_conv3) + b_conv3) # output size 32x32x128
h_pool3 = max_pool_2x2(h_conv3)                          # output size 13*13x256
h_pool3_drop = tf.nn.dropout(h_pool3, keep_prob)

# conv4 layer ##
W_conv4 = weight_variable([3 ,3 , 256, 512]) # patch 5x5, in size 128, out size 256
b_conv4 = bias_variable([512])
h_conv4 = tf.nn.relu(conv2d(h_pool3, W_conv4) + b_conv4) # output size 12x12x256
h_pool4 = max_pool_2x2(h_conv4)                          # output size 4x4x256
h_pool4_drop = tf.nn.dropout(h_pool4, keep_prob)

# conv5 layer ##
W_conv5 = weight_variable([3 ,3 , 512, 512]) # patch 5x5, in size 128, out size 256
b_conv5 = bias_variable([512])
h_conv5 = tf.nn.relu(conv2d(h_pool4, W_conv5) + b_conv5) # output size 12x12x256
h_pool5 = max_pool_2x2(h_conv5)                          # output size 4x4x256
h_pool5_drop = tf.nn.dropout(h_pool5, keep_prob)

## fc1 layer ##
W_fc1 = weight_variable([image_cnn_shape*512, 1024])
b_fc1 = bias_variable([1024])
#[n_samples, 4*4*64]
h_pool_flat = tf.reshape(h_pool5_drop, [-1, image_cnn_shape*512])
h_fc1 = tf.nn.relu(tf.matmul(h_pool_flat, W_fc1) + b_fc1)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

## fc2 layer ##
W_fc2 = weight_variable([1024, 512])
b_fc2 = bias_variable([512])
h_fc2 = tf.nn.relu(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)
h_fc2_drop = tf.nn.dropout(h_fc2, keep_prob)

## class layer ##
W_fc3 = weight_variable([512, 15])
b_fc3 = bias_variable([15])
output = tf.matmul(h_fc2_drop, W_fc3) + b_fc3
pre = tf.nn.softmax(output)

# the error between prediction and real data
#cross_entropy = tf.reduce_mean(-tf.reduce_sum(ys * tf.log(prediction), reduction_indices=[1]))       # loss
#train_step = tf.train.AdamOptimizer(learning_rate=lr).minimize(cross_entropy)

with tf.name_scope('cross_entropy'):
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=output, labels=ys))
    #loss = tf.reduce_mean(-tf.reduce_sum(ys * tf.log(pre), reduction_indices=[1]))

with tf.name_scope('accuracy'):
    correct_prediction = tf.equal(tf.argmax(tf.nn.softmax(output),1), tf.argmax(ys,1)) #如果答案對則回傳true
    compute_acc = tf.reduce_mean(tf.cast(correct_prediction, tf.float32)) #將回傳的true/false轉乘1/0並計算平均(計算正確率)

with tf.name_scope('train'):
    #使用adam做optimization最小化loss funciotn(不斷取微分並逼近local min)
    train_step = tf.train.AdamOptimizer(learning_rate=lr).minimize(loss)

In [7]:
import tensorflow as tf
sess = tf.Session()
saver = tf.train.Saver()
sess.run(tf.global_variables_initializer())
saver.restore(sess, "model_cnn2/model.ckpt")

for j in tqdm(range(0, 1500)):
    batch_idx_start = j * 1
    batch_idx_stop = (j+1) * 1
    x_batch = test[batch_idx_start : batch_idx_stop]
    
    result = sess.run(pre, feed_dict = {xs: x_batch, keep_prob: 1})
    test_submit_df["class"][j] = np.argmax(result)
    print(result)
    print(test_submit_df["class"][j])
#result = sess.run(pre, feed_dict={xs: test, keep_prob: 1})
#print("np.argmax(result)=" + np.argmax(result))

INFO:tensorflow:Restoring parameters from model_cnn2/model.ckpt


  0%|          | 0/1500 [00:00<?, ?it/s]/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
  0%|          | 3/1500 [00:01<13:14,  1.88it/s]

[[  7.22573176e-02   1.76550731e-25   1.39310304e-03   3.69179123e-12
    2.00174172e-18   3.44994009e-01   2.25282626e-14   5.81355512e-01
    2.51876575e-11   1.92653616e-22   1.42333356e-20   1.61375739e-11
    7.45124709e-08   1.95102748e-18   1.22944661e-18]]
7
[[  1.99500157e-19   1.96768159e-24   1.41338348e-20   9.92248511e-30
    1.55900143e-38   0.00000000e+00   1.47712426e-10   0.00000000e+00
    4.63103155e-22   0.00000000e+00   9.64008033e-01   2.33452499e-38
    1.22565257e-27   0.00000000e+00   3.59919742e-02]]
10
[[  4.43142517e-06   4.89094569e-15   1.21251764e-02   4.28784051e-06
    5.60588407e-18   9.87470210e-01   5.06877115e-16   6.62165381e-15
    3.32477562e-13   5.77156161e-13   2.36041051e-23   7.02054240e-06
    3.82113678e-04   6.75414640e-06   2.23262787e-15]]
5


  0%|          | 5/1500 [00:01<08:39,  2.88it/s]

[[  3.42715960e-11   8.98957126e-17   1.04089668e-05   2.95194258e-17
    4.06291807e-19   9.98401344e-01   4.63425829e-15   5.16117110e-18
    1.58824562e-03   1.46341188e-14   1.47456980e-20   2.04796288e-21
    3.57597618e-09   1.23672307e-12   1.97878248e-23]]
5
[[  6.76642978e-11   7.43840685e-14   6.23195301e-05   7.83639651e-21
    4.25386333e-23   9.99814570e-01   1.37905811e-12   2.03617098e-14
    2.59085510e-07   6.43999436e-22   8.65399188e-24   3.33951582e-14
    1.22801852e-04   4.79233231e-09   1.72500447e-09]]
5
[[  1.90554983e-13   5.28905583e-15   1.50124801e-09   5.67082975e-20
    1.44012740e-17   2.91017987e-10   1.27265102e-15   9.99975085e-01
    2.34234285e-05   4.20904055e-15   4.05172467e-12   3.16063849e-22
    1.52851703e-06   4.95238830e-16   8.12482720e-12]]
7


...

[[  1.10527303e-08   2.21435741e-08   1.00820166e-07   1.40863571e-11
    2.59767541e-19   9.99651670e-01   4.23102949e-17   6.00443445e-15
    1.87148660e-14   1.03241532e-15   9.84732039e-23   5.23367447e-08
    3.48190690e-04   7.28581689e-16   1.31400155e-16]]
5
[[  7.30105400e-01   1.73824089e-21   6.59941215e-05   9.89144580e-20
    1.50145365e-16   2.14082867e-01   2.16179585e-09   5.54987271e-11
    5.57454303e-02   2.00235661e-13   2.00653747e-11   1.57002959e-12
    3.26049673e-07   1.38790716e-17   8.25998820e-18]]
0
[[  7.72311270e-10   1.77036069e-18   1.47166511e-05   1.51570136e-20
    1.98708288e-14   1.15802287e-07   8.88774480e-07   4.02987504e-10
    9.70264137e-01   5.36789890e-10   1.46756361e-06   1.69515832e-10
    2.97186356e-02   1.14593057e-09   7.87541758e-15]]
8
[[  4.80761573e-22   1.99872126e-23   3.53418358e-12   1.53531719e-22
    1.95091514e-25   5.00137592e-16   8.09576741e-05   1.79027486e-24
    9.99919057e-01   1.46530215e-20   8.51060986e-17   5.56

100%|█████████▉| 1495/1500 [01:35<00:00, 15.68it/s]

[[  1.26590024e-07   4.02239368e-22   1.91094660e-08   1.67679517e-23
    8.32667798e-17   1.45431049e-03   1.63143798e-09   9.89497185e-01
    2.64587311e-06   7.13129371e-13   5.10510017e-06   4.11069448e-19
    9.04062204e-03   2.45329879e-14   1.06200620e-20]]
7
[[  9.93862629e-01   1.26610178e-09   8.62464162e-08   2.60210575e-10
    8.60936838e-24   2.76448175e-09   5.93960145e-03   9.35371727e-26
    1.85736212e-06   1.08270136e-18   1.95736066e-04   3.27300465e-10
    3.72500120e-09   1.71117973e-29   1.95837121e-13]]
0
[[  1.05903595e-17   1.62943590e-15   6.09106099e-11   2.43812318e-14
    1.47674435e-18   2.15289006e-20   2.11645229e-07   8.49752552e-18
    6.49511236e-08   1.35581869e-19   2.40162387e-03   4.32504873e-21
    1.29600384e-11   4.85943340e-18   9.97598112e-01]]
14
[[  1.68523155e-15   5.07937700e-17   5.28585136e-01   2.30216461e-15
    1.14732407e-20   1.53011367e-10   7.86638143e-15   9.09120002e-11
    4.76734314e-08   1.47877379e-19   1.01985778e-16   4.0

100%|██████████| 1500/1500 [01:35<00:00, 15.69it/s]

[[  8.37398103e-20   2.85908597e-21   3.47756391e-16   6.81996651e-17
    9.99177224e-04   9.21206670e-16   9.68784511e-01   6.66360298e-26
    5.50752855e-04   3.25965972e-17   2.96655595e-02   1.92192675e-19
    1.11510616e-12   1.05496016e-12   4.02641903e-16]]
6
[[  1.09374701e-18   6.11712906e-29   3.35100871e-14   3.83934605e-31
    3.50704919e-32   3.61229879e-10   5.33185616e-28   1.00000000e+00
    8.14015179e-16   2.42807600e-32   2.45968491e-35   2.04615862e-32
    3.87005478e-13   5.12628024e-15   8.03011224e-30]]
7
[[  9.99980330e-01   2.62927195e-18   6.85130284e-13   9.93379528e-17
    7.20103840e-19   1.96250512e-05   2.01996020e-09   2.25776674e-15
    8.17121648e-11   8.82661995e-22   9.74585470e-14   1.16917931e-09
    4.41380230e-08   5.68264680e-14   1.52017067e-15]]
0
[[  4.88168110e-33   3.34799305e-33   2.32990970e-29   3.81159027e-31
    2.42476480e-30   5.83104119e-29   6.27964458e-10   6.37833292e-27
    1.00000000e+00   5.21264929e-31   3.57545660e-09   5.61

In [8]:
test_submit_df.sample(5)

,id,class
249,eaa0045c2d89fe3b67d6a1950d85a0126a80a57dba292c...,3
313,1881fb80f409ce93375642c791212b8fd259172315c0b2...,12
1032,8425e1d96c54173599c6e51915711bc800d6d3d6d6f3b5...,7
208,a47d5d045c7c93fd7f798fa4db96b2d3356feb724b9e32...,10
853,a6df5f87ba2e58e0ebb8d05bac62be420165e010eca46f...,8


In [9]:
#StackingSubmission = pd.DataFrame({ 'id': test_submit_df.id, 'class': test_submit_df.class})
test_submit_df.to_csv("submission.csv", index=False)
test_submit_df2 = pd.read_csv('submission.csv', encoding = "utf-8", dtype = {'type': np.int32})
test_submit_df2.head(5)

,id,class
0,c117693e1cf24a5232090d1548cb11d4e5ea0df65680c4...,7
1,96baacc2e97886a998807ce197574821a6dc83c227c746...,10
2,f9bd26db23eb9b544ca78be79b11b4d1259e802885861d...,5
3,b42bcd8e6645fcc2ac40ee44b7dc8d74a77081d0aea7a1...,5
4,5bc53cef9168882f0ff67a81b3e7269f62b7fd5343d06d...,5
